# Notebook 01 - Importação e Limpeza de Dados

#### Inicialmente, iremos importar o dataset .csv para um dataframe Pandas e depois salvar os dados no banco de dados PostgreSQL, para simular um cenário realista de bancos, os quais guardam os dados de seus clientes em bancos de dados relacionais.

1. Importação das bibliotecas necessárias e leitura do dataset .csv em um dataframe Pandas.

In [3]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

data = pd.read_csv('../data/german_credit_data.csv')
data.head()


,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


2. Adicionar coluna de salário anual fictício.

In [4]:
import numpy as np

def estimar_salario(job_level):
    if job_level == 0:
        return np.random.uniform(15000, 22000) # R$ anual
    elif job_level == 1:
        return np.random.uniform(25000, 40000)
    elif job_level == 2:
        return np.random.uniform(45000, 80000)
    elif job_level == 3:
        return np.random.uniform(90000, 180000)
    else:
        return np.random.uniform(30000, 50000)

In [5]:
data['salario_anual'] = data['Job'].apply(estimar_salario) # Aplicar a função estimar_salario numa coluna nova 'salario_anual' usando 'Job' como referência
data['salario_anual'] = data['salario_anual'].round(2) # Arredonda para 2 casas decimais para deixar semelhante ao dinheiro real

data = data.drop(columns=['Unnamed: 0', 'Job'], axis=1) # Remove colunas desnecessárias

dicionario = { # Cria um dicionário para poder renomear as colunas com base no nome das colunas das tabelas do banco de dados
    'Age': 'idade',
    'Sex': 'sexo',
    'Housing': 'situacao_moradia',
    'Saving accounts': 'status_poupanca',
    'Checking account': 'status_conta_corrente',
    'Credit amount': 'valor_emprestimo',
    'Duration': 'prazo_meses',
    'Purpose': 'finalidade_emprestimo',
    'Risk': 'status_inadimplencia'
}

data = data.rename(columns=dicionario) # Renomeia as colunas do dataframe

data['status_inadimplencia'] = data['status_inadimplencia'].map({'good': 0, 'bad': 1}) # converte valores good e bad para 0 e 1
data.head()

,idade,sexo,situacao_moradia,status_poupanca,status_conta_corrente,valor_emprestimo,prazo_meses,finalidade_emprestimo,status_inadimplencia,salario_anual
0,67,male,own,NaN,little,1169,6,radio/TV,0,77822.93
1,22,female,own,little,moderate,5951,48,radio/TV,1,47114.21
2,49,male,own,little,NaN,2096,12,education,0,28371.53
3,45,male,free,little,little,7882,42,furniture/equipment,0,72152.17
4,53,male,free,little,little,4870,24,car,1,59938.93
